# Sentinel 2 Atmospheric Correction in Google Earth Engine

Sentinel 2 is a satellite mission from the European Space Agency (ESA). It is very similar to NASA's Landsat satellites.

There are two Sentinel 2 satellites, orbiting in a twin configuration, that provides 5 day revisits at the equator at up to 10m per pixel!

Atmospheric correction requires a desktop application at present (i.e. ESA do not just provide the data corrected already). Here is a way to correct the data in Google Earth Engine

### Start Earth Engine

In [3]:
import os, sys
basepath = os.path.dirname(os.getcwd())
sys.path.append(os.path.join(basepath,'bin'))
import ee
ee.Initialize()

### Find Some Assets

We need a collection of images to correct. Let's take a look at beautiful Hawaii using the Sentinel 2 satellites.

In [5]:
Honolulu = ee.Geometry.Point(-157.8392, 21.306)

startDate = ee.Date('2016-01-01')
endDate = ee.Date('2016-06-01')

ic = ee.ImageCollection('COPERNICUS/S2')\
    .filterBounds(Honolulu)\
    .filterDate(startDate, endDate)\
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))\
    .filter(ee.Filter.lt('MEAN_SOLAR_ZENITH_ANGLE',60))
#                      ^
#                      |
# here we filtered out cloudy images and high solar zenith angles,
# i.e. low solar elevation angles

### Find Atmospheric Inputs

Atmospheric correction code use the state of the atmosphere (e.g. H20, O3, AOT, etc.) as input. We could find that information and store it in a feature collection (i.e. whose properties are the atmospheric parameters)

In [34]:
import atmcorr
import importlib
importlib.reload(atmcorr)
Atmcorr = atmcorr.Atmcorr

from atmcorr import Atmcorr
input_fc = Atmcorr.findInputs(ic)

lets take a look..

In [28]:
print(input_fc.first().getInfo())

{'type': 'Feature', 'geometry': {'geodesic': True, 'type': 'Point', 'coordinates': [-157.5141698587372, 21.024068719262132]}, 'id': '20160411T210929_20160411T223342_T04QFJ', 'properties': {'H2O': 2.9399999618530273, 'altitude': {'altitude': 0.0}, 'AOT': 0.18000000715255737, 'O3': 0.281, 'solar_z': 126.95980770219468}}


We need to access this data *locally* (i.e. on the client-side) to run our atmospheric correction code..

However, using getInfo() multiple times is not a good batch processing strategy. One solution is to export this information all in one go to a .csv file.

In [35]:
task = Atmcorr.exportInputs(input_fc)
task.start()

---

.. Task Running ...

Once complete, download the file from your Google Drive. 

(**tip**) this repo will ignore anything inside the directory called 'files/', so that is a good place to put data (i.e. without them floating about in repo history)

---

Here is where I put my atmospheric correction input file..

In [36]:
#A suggested place to download the input file..
atmcorr_input_filepath = os.path.join(basepath,'files','CSV','atmcorr_inputs.csv')
print(atmcorr_input_filepath)

/home/sam/git/github/gee-atmcorr-S2-batch/files/CSV/atmcorr_inputs.csv


### Option 1: 6S Radiative Transfer Code

For a small number of locations (as in this example) we could run the 6S radiative transfer code in a *reasonable* amount of time (i.e. ~ less than 30 secs):

In [37]:
# Blue 
Atmcorr.run6S(atmcorr_input_filepath, predefined_wavelength = 'S2A_MSI_02')

# Green 
Atmcorr.run6S(atmcorr_input_filepath, predefined_wavelength = 'S2A_MSI_03')

# Red 
Atmcorr.run6S(atmcorr_input_filepath, predefined_wavelength = 'S2A_MSI_04')

# Near-infrared 
Atmcorr.run6S(atmcorr_input_filepath, predefined_wavelength = 'S2A_MSI_08')

running 6S.. (this may take a while..)
time check 15.043583154678345
running 6S.. (this may take a while..)
time check 8.499802589416504
running 6S.. (this may take a while..)
time check 8.556232452392578
running 6S.. (this may take a while..)
time check 8.421323299407959


This will update the .csv file. For example:

In [38]:
import pandas as pd
updated_csv = pd.read_csv(atmcorr_input_filepath)
print(updated_csv.keys())

Index(['AOT', 'H2O', 'O3', 'altitude', 'solar_z', 'geometry',
       'S2A_MSI_02_6S_a', 'S2A_MSI_02_6S_b', 'S2A_MSI_03_6S_a',
       'S2A_MSI_03_6S_b', 'S2A_MSI_04_6S_a', 'S2A_MSI_04_6S_b',
       'S2A_MSI_08_6S_a', 'S2A_MSI_08_6S_b'],
      dtype='object')


### Option 2: 6S Emulator

Running 6S code can take a **long** time. Here is where the 6S emulator comes in..

---
Its based on some awesome interpolated lookup tables..

Here is where you can [download](https://drive.google.com/open?id=0B9tjEx6g_uGQT1JhZ3hGcENySWc) some for Sentinel 2

---

While that downloads..

(explain directory tree)
(and link to how people can build their own iLUTs)

In [8]:
# The iLUT files are organized in the following directory tree
iLUT_relative_path = os.path.join('iLUTs','S2A_MSI','Continental','view_zenith_0')
print('iLUT_relative_path:\n../'+iLUT_relative_path)

iLUT_relative_path:
../iLUTs/S2A_MSI/Continental/view_zenith_0


In [9]:
# This is where I placed the files
iLUT_path = os.path.join(basepath,'files',iLUT_relative_path)
print('Example iLUT_path:\n'+iLUT_path)

Example iLUT_path:
/home/sam/git/github/gee-atmcorr-S2-batch/files/iLUTs/S2A_MSI/Continental/view_zenith_0


In [30]:
import atmcorr
import importlib
importlib.reload(atmcorr)
run6SE = atmcorr.Atmcorr.run6SE

for bandnum in ['02','03','04','08']:
    ilut_name = 'S2A_MSI_'+bandnum+'.ilut'
    iLUT_filepath = os.path.join(iLUT_path,ilut_name)
    run6SE(atmcorr_input_filepath, iLUT_filepath)

Upload to fusion table

Use in GEE

own target sites

In [ ]:
#target_locations = fc
#assetIDs = Testsites().get(target_locations)

then same as before..

In [ ]:
#assetIDs_with_inputs = Atmcorr.findInputs(assetIDs)
#task = Atmcorr.exportInputs(assetIDs_with_inputs).start()
#Atmcorr.run6S('newsites.csv', predefined = 'S2A_MSI_02')